In [60]:
import imutils
import cv2
import numpy as np

def detectLetters(image,kernelSize,rtvMode,approx):
    boundRect = [None]
    #image = cv2.imread(inputFilename)
    
 
    # convert the image to grayscale, blur it, and find edges
    # in the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)
 
    # show the original image and the edge detected image
    cv2.imwrite('originals/dltme1.png',edged)
    print ("STEP 1: Edge Detection")

    #Binary Dialation
    kernelSize = (17,3)
    element = cv2.getStructuringElement (cv2.MORPH_RECT, kernelSize )
    img_threshold = cv2.morphologyEx(edged,cv2.MORPH_CLOSE,element)
    cv2.imwrite('originals/dltme2.png',img_threshold)
    print('Binary dialation done...')
    
    #Find Contours
    im2,contours,hierarchy = cv2.findContours(img_threshold,rtvMode,approx)
    contours_poly = [None]*len(contours)
    for i in range(0,len(contours)):
        if len(contours[i])>100:            
            contours_poly[i]=(cv2.approxPolyDP(contours[i], 3, True))            
            x,y,w,h = cv2.boundingRect(contours_poly[i])            
            if w>h:
                temp = x,y,w,h
                boundRect.append(temp)
    return boundRect

def extractTextArea(inputFile,outputFile,kernelSize,rtvMode,approx):
    #implementation    
    img = cv2.imread(inputFile)
    ratio = img.shape[0] / 500.0
    orig = img.copy()
    img = imutils.resize(img, height = 500)
    letterBBoxes=detectLetters(img,kernelSize,rtvMode,approx)
    for i in range(0,len(letterBBoxes)-1):
        if letterBBoxes[i] != None:        
            temp = letterBBoxes[i]
            cv2.rectangle(orig,(temp[0],temp[1]),(temp[0]+temp[2],temp[1]+temp[3]),(0,255,0),3)
            crop_image = img[temp[1]:temp[1]+temp[3], temp[0]:temp[0]+temp[2]]
            filename = 'originals/TextArea/'+inputFile[10:-4]+'_'+str(i)+'.png'
            #print(filename)            
            cv2.imwrite(filename,crop_image)
            #print('cropped image created')

    cv2.imwrite(outputFile,img)
    


In [61]:
kernelSize = (17,3)
extractTextArea('originals/Pancard.png','originals/Pancard_1.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

kernelSize = (17,3)
extractTextArea('originals/Pancard_Web_1.png','originals/Pancard_Web_1_1.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

kernelSize = (17,3)
extractTextArea('originals/Pancard_Web_2.png','originals/Pancard_Web_2_1.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

kernelSize = (17,3)
extractTextArea('originals/Pancard_Web_3.png','originals/Pancard_Web_3_1.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

kernelSize = (17,3)
extractTextArea('originals/Pancard_Web_4.png','originals/Pancard_Web_4_1.png',kernelSize,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
print('all done...')

STEP 1: Edge Detection
Binary dialation done...
STEP 1: Edge Detection
Binary dialation done...
STEP 1: Edge Detection
Binary dialation done...
STEP 1: Edge Detection
Binary dialation done...
STEP 1: Edge Detection
Binary dialation done...
all done...


In [37]:
import pickle
import cv2
from PIL import Image
import numpy as np
import imutils

def setImage(image):
    gray_masked = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    (thresh, im_bw) = cv2.threshold(gray_masked, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    #print(im_bw.shape)
    #Draw a white canvas
    white_img = np.zeros((40,40), np.uint8)
    white_img[:,:] = (0)
    
    #Paste the image onto the white canvas
    x_offset=y_offset=1
    white_img[y_offset:y_offset+image.shape[0], x_offset:x_offset+image.shape[1]] = im_bw
    
    return white_img
    
def split(image):
    image = image.reshape(1,1600)
    predicted = loaded_model.predict(image)
    if predicted == 1:
        return True
    else:
        return False
    
loaded_model = pickle.load(open('Model output/CharSplit.sav', 'rb'))
org_img = cv2.imread('originals/TextArea/Pancard_10.png')
r = 200.0 / org_img.shape[1]
dim = (200, int(org_img.shape[0] * r))
 
# perform the actual resizing of the image and show it
resized = cv2.resize(org_img, dim, interpolation = cv2.INTER_AREA)
#resized = org_img

#print(org_image)
(height, width,_) = resized.shape
init_column = 0

i=0
j=1
end_column = 14
while (init_column < width):
    i=i+1
    
    temp_image = resized[:,init_column:init_column+end_column+i,:]
    
    if split(setImage(temp_image)):                
        cv2.imwrite('dltme_'+str(j)+'_m.png',temp_image)
        #cv2.imwrite('dltme_'+str(j)+'_a.png',setImage(temp_image))
        j=j+1
        init_column = init_column+end_column+i
        i=0
    elif (init_column+end_column+i >= 40):
        init_column = init_column+1
        i=0
    elif (init_column+end_column+i >= width):
        init_column = width
    
print('images printed')

images printed
